In [24]:
import os
import cv2
import numpy as np

In [49]:
file_dir = "C:\\Users\\caiosmedeiros\\Downloads\\PKLot\\PKLot\\PKLot\\UFPR04"
weather_folders = os.listdir(file_dir)
file_paths = []

for weather_folder in weather_folders:
    complete_path = os.path.join(file_dir, weather_folder)
    dates = os.listdir(complete_path)
    for date in dates:
        path = os.path.join(complete_path, date)
        files = [os.path.join(path, os.listdir(path)[i]) for i in range(len(os.listdir(path))) if 'jpg' in os.listdir(path)[i]]
        file_paths.extend(files)

In [55]:
zerada = np.zeros((720, 1280), dtype=np.uint8)
def show(img):
    cv2.imshow("Imagem", img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
show(img)

In [163]:
from ultralytics import YOLO

model = YOLO(model="yolov8n.pt")
def get_bounding_boxes(img):
    results = model.predict(img)
    coordinates = []
    for result in results:
        for box in result.boxes:
            if box[0].cls==2:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                w = x2-x1
                h = y2-y1
                center = [int(x1+(w/2)), int(y1+(h/2))]
                coordinates.append(center)
    return coordinates

In [202]:
centers = []
for file_path in file_paths[0:50]:
    img = cv2.imread(file_path)
    coordinates = get_bounding_boxes(img)
    centers.extend(coordinates)


0: 384x640 52 cars, 2 trucks, 161.3ms
Speed: 4.9ms preprocess, 161.3ms inference, 3.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 53 cars, 1 truck, 527.4ms
Speed: 2.9ms preprocess, 527.4ms inference, 10.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 54 cars, 1 truck, 357.7ms
Speed: 4.2ms preprocess, 357.7ms inference, 5.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 51 cars, 1 bus, 1 truck, 792.2ms
Speed: 20.3ms preprocess, 792.2ms inference, 3.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 50 cars, 1 truck, 226.5ms
Speed: 5.8ms preprocess, 226.5ms inference, 4.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 52 cars, 1 bus, 327.3ms
Speed: 4.0ms preprocess, 327.3ms inference, 4.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 50 cars, 1 truck, 214.7ms
Speed: 4.3ms preprocess, 214.7ms inference, 4.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 55 cars

In [211]:
img = cv2.imread(file_paths[10])

In [204]:
for point in centers:
    cv2.ellipse(img, point, angle=0, startAngle=0, endAngle=0, color=(255,0,0), axes=(5,5), thickness=2)

In [212]:
show(img)

In [158]:
centers

[[840, 370],
 [816, 437],
 [722, 622],
 [536, 607],
 [868, 321],
 [1204, 353],
 [562, 290],
 [39, 336],
 [137, 274],
 [998, 544],
 [892, 224],
 [784, 518],
 [1023, 423],
 [1187, 294],
 [215, 229],
 [976, 657],
 [1036, 340],
 [1019, 102],
 [95, 192],
 [610, 237],
 [882, 272],
 [511, 342],
 [1045, 279],
 [1202, 608],
 [728, 313],
 [655, 193],
 [387, 286],
 [916, 191],
 [928, 161],
 [535, 176],
 [480, 205],
 [351, 234],
 [285, 187],
 [687, 158],
 [67, 532],
 [967, 82],
 [440, 225],
 [776, 228],
 [444, 244],
 [809, 93],
 [917, 184],
 [833, 161],
 [636, 109],
 [381, 121],
 [249, 626],
 [560, 149],
 [303, 280],
 [480, 146],
 [422, 101],
 [400, 205],
 [221, 132],
 [562, 98],
 [840, 371],
 [723, 621],
 [817, 438],
 [1203, 352],
 [137, 274],
 [39, 336],
 [535, 606],
 [562, 290],
 [998, 544],
 [868, 321],
 [511, 342],
 [1202, 607],
 [785, 518],
 [1023, 423],
 [892, 225],
 [976, 657],
 [95, 192],
 [1036, 340],
 [214, 229],
 [1191, 292],
 [882, 271],
 [1018, 102],
 [610, 237],
 [351, 233],
 [741, 

In [213]:
from sklearn.cluster import DBSCAN
import random

# Convert the list of centers into a numpy array
X = np.array(centers)

# Run DBSCAN with adjusted parameters (eps defines the maximum distance for neighboring points)
db = DBSCAN(eps=20, min_samples=20).fit(X)
labels = db.labels_

# Calculate the number of clusters (excluding noise, which is labeled as -1)
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
n_noise_ = list(labels).count(-1)
print("Estimated number of clusters: %d" % n_clusters_)
print("Estimated number of noise points: %d" % n_noise_)

# Create a copy of the original image to draw the clusters
img_clustered = img.copy()

# Generate random colors for each cluster (skip the noise label)
colors = {}
for label in set(labels):
    if label == -1:
        continue  # Skip outliers
    colors[label] = (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))

# Draw circles on the image for each center that is not an outlier
for (center, label) in zip(X, labels):
    if label == -1:
        continue  # Do not draw outliers
    cv2.circle(img_clustered, (center[0], center[1]), 10, colors[label], thickness=-1)

# Display the image with the clusters overlaid
cv2.imshow("Clusters on the image", img_clustered)
cv2.waitKey(0)
cv2.destroyAllWindows()

Estimated number of clusters: 54
Estimated number of noise points: 233
